In [265]:
import requests
import re
import pandas as pd
import time
from bs4 import BeautifulSoup

# Get all the symbols

In [81]:
response = requests.get("https://coinmarketcap.com/currencies/")
soup = BeautifulSoup(response.text, 'html.parser')

In [84]:
main_table = [t for t in soup.find_all('table') if t.has_attr('id') and t['id'] == 'currencies']
assert len(main_table) == 1, "Only one main table"
main_table = main_table[0]

In [108]:
coins = [t for t in main_table.find_all('tr') if t.has_attr('id')]
coin_aliases = [re.search("\/currencies\/(.*)\/", t.find_all('a')[0]['href']).group(1) for t in coins]
coin_aliases

['bitcoin',
 'ethereum',
 'ripple',
 'litecoin',
 'ethereum-classic',
 'dash',
 'nem',
 'monero',
 'iota',
 'bitconnect',
 'stratis',
 'zcash',
 'bitshares',
 'bytecoin-bcn',
 'antshares',
 'steem',
 'waves',
 'qtum',
 'siacoin',
 'dogecoin',
 'stellar',
 'byteball',
 'lisk',
 'factom',
 'decred',
 'digibyte',
 'gamecredits',
 'pivx',
 'komodo',
 'bancor',
 'nxt',
 'chaincoin',
 'bitcoindark',
 'leocoin',
 'lykke',
 'decent',
 'peercoin',
 'ark',
 'syscoin',
 'emercoin',
 'nexus',
 'verge',
 'reddcoin',
 'ubiq',
 'asch',
 'namecoin',
 'peerplays-ppy',
 'gulden',
 'monacoin',
 'cloakcoin',
 'daxxcoin',
 'counterparty',
 'dubaicoin-dbix',
 'library-credit',
 'omni',
 'e-dinar-coin',
 'skycoin',
 'sibcoin',
 'bitbay',
 'blocknet',
 'ybcoin',
 'potcoin',
 'crown',
 'zcoin',
 'viacoin',
 'groestlcoin',
 'elastic',
 'blackcoin',
 'burst',
 'mooncoin',
 'iocoin',
 'golos',
 'vertcoin',
 'faircoin',
 'firstcoin',
 'voxels',
 'novacoin',
 'mysterium',
 'salus',
 'monetaryunit',
 'nav-coin',
 'd

# Get all the market prices

In [274]:
def get_pair_info(tr_text):
    """
    Get pair info from html text of <tr>
    """
    tds = tr_text.find_all('td')
    exchange = tds[1].text
    pair = tds[2].text
    volume = float(tds[3].find('span')['data-usd']) if tds[3].find('span')['data-usd'] != '?' else 0
    price = float(tds[4].find('span')['data-usd']) if tds[4].find('span')['data-usd'] != '?' else 0
    share = float(tds[5].text[0:-1])
    time = tds[6].text

    return exchange, pair, volume, price, share, time

In [263]:
def get_arb_prices(coin_alias, tail_num=5):
    """
    Get the highest and lowest 5 prices among all exchanges.
    """
    # Get the market html
    coin_market_ret = requests.get("https://coinmarketcap.com/currencies/%s/#markets" % coin_alias)
    assert coin_market_ret.status_code == 200, "Status code (%d) is not 200.\n%s" % (coin_market_ret.status_code, coin_market_ret.text)
    coin_market_soup = BeautifulSoup(coin_market_ret.text, 'html.parser')
    coin_table = [t for t in coin_market_soup.find_all('table') if t.has_attr('id') and t['id'] == 'markets-table']
    assert len(coin_table) == 1, "Only one main table"
    coin_table = coin_table[0]
    # Get all the records
    coin_records = coin_table.find_all('tr')
    records = pd.DataFrame(columns=['exchange', 'pair', 'volume', 'price', 'share', 'last_updated'],
                           data=[get_pair_info(t) for t in coin_records 
                                 if len(t.find_all('td')) > 0])
    # Take only recent records
    records = records[(records['last_updated'] == "Recently") & (records['price'] > 0)]
    # Get the lowest and highest records
    lowest = records.sort_values("price").head(tail_num).apply(lambda x: "%s (%.6f)" % (x["exchange"], x["price"]), axis=1)
    highest = records.sort_values("price", ascending=False).head(tail_num).apply(lambda x: "%s (%.6f)" % (x["exchange"], x["price"]), axis=1)
    return highest, lowest

In [286]:
level = 3
arb_pairs = pd.DataFrame(columns=["h%d" % (t+1) for t in range(0,level)]+["l%d" % (t+1) for t in range(0,level)])
for coin_alias in coin_aliases[0:50]:
    highest, lowest = get_arb_prices(coin_alias)
    highest = (highest.tolist() + ['???'] * level)[0:level]
    lowest = (lowest.tolist() + ['???'] * level)[0:level]
    arb_pairs.loc[coin_alias] = highest + lowest
    time.sleep(5)
arb_pairs

,h1,h2,h3,l1,l2,l3
bitcoin,Cryptopia (2784.780000),Bittrex (2679.840000),Luno (2545.400000),LocalTrade (215.452000),LocalTrade (218.400000),Bitcoin Growth Fund Exchange (1669.730000)
ethereum,Decentrex (1170520.000000),Livecoin (24787.200000),LocalTrade (1856.830000),CryptoDerivatives (4.649300),Quoine (9.774240),Quoine (10.730000)
ripple,Cryptomate (0.204617),Rippex (0.198937),BTC Markets (0.195725),Mr. Ripple (0.164611),Mr. Ripple (0.164835),Bitstamp (Ripple Gateway) (0.169905)
litecoin,Cryptopia (4178130.000000),CoinGather (1193750.000000),Novaexchange (640.551000),Novaexchange (1.540910),Cryptopia (2.901570),DCExchange (4.250000)
ethereum-classic,CoinExchange (20.367800),C-CEX (18.950000),Cryptomate (18.147400),CoinExchange (13.331200),CoinExchange (14.916400),HitBTC (15.426000)
dash,Novaexchange (4019.090000),Novaexchange (1392.610000),LocalTrade (546.953000),OpenLedger DEX (0.122324),OpenLedger DEX (4.937730),Crypto Dao (17.766900)
nem,Cryptopia (0.190386),Cryptopia (0.125461),LiteBit.eu (0.122334),BTER (0.105511),BTER (0.110284),Cryptopia (0.112027)
monero,Cryptomate (37.752500),Cryptopia (36.342700),Poloniex (35.176600),Poloniex (30.023900),BTC Alpha (32.706400),BTER (32.814700)
iota,IOTA Exchange (0.174610),Bitfinex (0.161823),Bitfinex (0.161727),Bitfinex (0.160470),Bitfinex (0.161727),Bitfinex (0.161823)
bitconnect,Novaexchange (139.575000),Livecoin (53.388900),BCC Exchange (53.377800),Novaexchange (52.638500),CoinExchange (53.201900),BCC Exchange (53.377800)


In [288]:
arb_pairs[['h3', 'l3']]

,h3,l3
bitcoin,Luno (2545.400000),Bitcoin Growth Fund Exchange (1669.730000)
ethereum,LocalTrade (1856.830000),Quoine (10.730000)
ripple,BTC Markets (0.195725),Bitstamp (Ripple Gateway) (0.169905)
litecoin,Novaexchange (640.551000),DCExchange (4.250000)
ethereum-classic,Cryptomate (18.147400),HitBTC (15.426000)
dash,LocalTrade (546.953000),Crypto Dao (17.766900)
nem,LiteBit.eu (0.122334),Cryptopia (0.112027)
monero,Poloniex (35.176600),BTER (32.814700)
iota,Bitfinex (0.161727),Bitfinex (0.161823)
bitconnect,BCC Exchange (53.377800),BCC Exchange (53.377800)
